# Assignment 1

## Exercise 4

### Instructions

Consider the FrozenLake8x8-v1 environment. It is similar to the FrozenLake-v1 that was studied
in Tinkering Notebook 2, but it consist of an 8 × 8 grid and thus have 64 states.

Write a code that find an optimal policy π∗(s) and the corresponding value function v∗(s).

In the quizz on you will be asked for example ”Which of these are optimal actions in state s = 26?”
or ”What is v∗(26)?”. So make sure that you can easily run code that can answer these types of
questions for different states.

Hint: You can check that your code seems to be working by ensuring that you get to correct answer
to the following:

- For the optimal policy v∗(26) = 0.80 (rounded to two decimals).
- In s = 26 the optimal action is 0 (left).


In [35]:
# Packages needed for this notebook
import gymnasium as gym
import gym_RLcourse
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output # Used to clear the ouput of a Jupyter cell.

In [36]:
class RandomAgent():
    
    def __init__(self, nA=4, nS=64):
        self.nA = nA # Number of actions
        self.nS = nS # Number of states
        
        # Uniform probabilities in each state.
        # That is, in each of the nS states
        # each of the nA actions has probability
        # 1/nA.
        self.probs = np.ones((nS,nA))/nA 

    def act(self, state):
        action = np.random.choice(self.nA, p=self.probs[state]) 
        return action # a random policy

In [37]:
def run_agent(env, agent):
    state, info = env.reset()
    time_step = 0
    total_reward = 0
    truncated = False
    terminated = False
    while not truncated and not terminated:
        action = agent.act(state);
        state, reward, terminated, truncated, info = env.step(action)
        total_reward += reward
        time_step += 1
        
        clear_output(wait=True)
        print("Time step:", time_step)
        print("State:", state)
        print("Action:", action)
        print("Total reward:", total_reward)
        
    if truncated:
        print("The environment was truncated even though a terminal state was not reached.")
    elif terminated:
        print("A terminal state was reached.")

In [38]:
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3

In [39]:
env = gym.make('FrozenLake8x8-v1', render_mode="human")
state, info = env.reset()
print("State space:", env.observation_space)
print("Action space:", env.action_space)

State space: Discrete(64)
Action space: Discrete(4)


In [40]:
agent = RandomAgent()
run_agent(env, agent)

Time step: 97
State: 41
Action: 1
Total reward: 0.0
A terminal state was reached.


In [19]:
env.close()

In [41]:
def compute_action_value(env, discount, s, a, v):
    
    action_value = 0
    
    # Loop through all possible (s', r) pairs
    for p, next_s, reward, _ in env.unwrapped.P[s][a]:
        action_value += p * (reward + discount * v[next_s])
    
    return action_value

In [42]:
def Bellman_RHS(env, discount, agent, s, v):
    
    state_value = 0
    
    for a in range(env.action_space.n):
        # Loop through all possible actions
        state_value += agent.probs[s][a] * compute_action_value(env, discount, s, a, v)
    
    return state_value

In [43]:
def Bellman_RHS_all(env, discount, agent, v0):
    # v0 is the given value function
    # v will be the right-hand side of the Bellman equation
    # If v0 is indeed the value function, then we should get v = v0.
    
    v = np.zeros(env.observation_space.n)
    
    for s in range(env.observation_space.n):
        v[s] = Bellman_RHS(env, discount, agent, s, v0)
    
    return v

In [44]:
def policy_evaluation(env, discount, agent, v0, max_iter=1000, tol=1e-6):
    
    v_old = v0
    
    for i in range(max_iter):
        v_new = Bellman_RHS_all(env, discount, agent, v_old)
        
        if np.max(np.abs(v_new-v_old)) < tol:
            break
            
        v_old = v_new
        
    return v_new

In [45]:
np.set_printoptions(precision=5, suppress=False)
env = gym.make('FrozenLake8x8-v1')
agent = RandomAgent()
discount = 1
# Write code for computing the state-value function
v0 = np.zeros((env.observation_space.n))
v = policy_evaluation(env, discount, agent, v0)
print(v.reshape(8,8))

[[1.88195e-03 2.14944e-03 2.79688e-03 4.10391e-03 6.53057e-03 9.78456e-03
  1.34274e-02 1.59485e-02]
 [1.61811e-03 1.77291e-03 2.14039e-03 2.98713e-03 5.70622e-03 9.39901e-03
  1.45526e-02 1.84735e-02]
 [1.20269e-03 1.18669e-03 1.00713e-03 0.00000e+00 3.91045e-03 7.55536e-03
  1.69135e-02 2.49228e-02]
 [8.05879e-04 7.66260e-04 7.02931e-04 7.70972e-04 2.38134e-03 0.00000e+00
  2.06255e-02 3.93840e-02]
 [4.50526e-04 3.71089e-04 2.68386e-04 0.00000e+00 4.84438e-03 1.15929e-02
  2.62057e-02 7.26052e-02]
 [1.75712e-04 0.00000e+00 0.00000e+00 1.44822e-03 5.40353e-03 1.53216e-02
  0.00000e+00 1.52227e-01]
 [7.70750e-05 0.00000e+00 1.09332e-04 3.89386e-04 0.00000e+00 4.42900e-02
  0.00000e+00 3.84076e-01]
 [5.57307e-05 3.45388e-05 4.79484e-05 0.00000e+00 5.39462e-02 1.61839e-01
  3.87280e-01 0.00000e+00]]


In [47]:
def greedy_policy(env, discount, agent, v):
    
    # The new policy will be a_probs
    # We start by setting all probabilities to 0
    # Then, when we have found the greedy action in a state, 
    # we change the probability for that action to 1.0.
    
    a_probs = np.zeros((env.observation_space.n, env.action_space.n)) 
    
    for s in range(env.observation_space.n):
        
        action_values = np.zeros(env.action_space.n)
        
        for a in range(env.action_space.n):
            # Compute action value for all actions
            action_values[a] = compute_action_value(env, discount, s, a, v)
            
        a_max = np.argmax(action_values) # A greedy action
        a_probs[s][a_max] = 1.0 # Always choose a greedy action!
        
    return a_probs

In [48]:
env = gym.make('FrozenLake8x8-v1', render_mode='human')
agent = RandomAgent()
discount = 1
# Enter code here
v_old = np.zeros(env.observation_space.n) 
for i in range(1000):
    v = policy_evaluation(env, discount, agent, v_old)

    if (np.max(np.abs(v-v_old))<1e-6):
        break

    v_old = v
    agent.probs = greedy_policy(env, discount, agent, v)

print(v.reshape(8,8))


[[0.99999 0.99999 1.      1.      1.      1.      1.      1.     ]
 [0.99999 0.99999 1.      1.      1.      1.      1.      1.     ]
 [0.99998 0.97819 0.92642 0.      0.85661 0.94623 0.98208 1.     ]
 [0.99997 0.93458 0.80107 0.4749  0.62362 0.      0.94468 1.     ]
 [0.99996 0.82559 0.54222 0.      0.53934 0.61119 0.85195 1.     ]
 [0.99996 0.      0.      0.16804 0.38321 0.44227 0.      1.     ]
 [0.99995 0.      0.19466 0.1209  0.      0.3324  0.      1.     ]
 [0.99995 0.73152 0.46309 0.      0.27747 0.55493 0.77747 0.     ]]


In [49]:
run_agent(env, agent)

Time step: 92
State: 63
Action: 2
Total reward: 1.0
A terminal state was reached.


In [50]:
env.close()